In [70]:
# basic imports
import numpy as np
import pandas as pd

from scipy import stats
import matplotlib.pyplot as plt

# for ML
from sklearn.metrics import mutual_info_score
from scipy.ndimage.interpolation import shift
from sklearn.neighbors import NearestNeighbors

# os library
from os.path import exists, join
from itertools import permutations

In [2]:
data_path = join(".", "data") if exists(join(".", "data")) \
    else join("..", "input", "statistical-learning-sapienza-spring-2021")

train = pd.read_csv(join(data_path, 'train.csv'), nrows=40)

def get_data(entry):
    id = entry.id
    variables = entry[['var1', 'var2', 'var3']].to_numpy()
    timeseries = entry[5 if "y" in entry.index else 4:].to_numpy(dtype=float).reshape((115, 116), order="F")
    if "y" in entry.index: return id, entry.y, variables, timeseries
    else: return id, variables, timeseries


timeseries = train.apply(lambda row: get_data(row)[-1], axis=1).to_list()

In [66]:
data = timeseries[0]

def lagged_timeseries(timeseries, steps):
    return list(map(lambda i: list(map(lambda j: shift(timeseries[:, i], j, cval=np.NaN), range(steps))), range(timeseries.shape[1])))

manifolds_list = list(map(lambda x: lagged_timeseries(x, 3), timeseries))

In [58]:
def get_symbols(timeseries, threshold=0.1):
    symbols_and_weights = []
    for column in range(timeseries.shape[1]):
        t = timeseries[:, column]
        change = shift(t, 1, cval=0) - t
        symbols = (abs(change)>=threshold)*np.sign(change)
        weights = (abs(change)-abs(change).min())/(abs(change).max()-abs(change).min())
        symbols_and_weights.append(list(zip(symbols, weights)))
    return symbols_and_weights

symbols_and_weights_list = list(map(lambda x: get_symbols(x, threshold=0.1), timeseries))

<ipython-input-58-243154958139>:7: RuntimeWarning: invalid value encountered in true_divide
  weights = (abs(change)-abs(change).min())/(abs(change).max()-abs(change).min())


In [ ]:
def compute_causalities(manifolds, symbols_and_weights, n_neighbors=6):
    n_features = len(manifolds[0])
    causality_matrix = np.zeros(shape=(n_features, n_features))
    permutations(range(manifolds))
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree').fit(manifolds[0])
    distances, indices = nbrs.kneighbors(X)
    indices

In [63]:
list(map(lambda i: compute_causalities(manifolds_list[i], symbols_and_weights_list[i]), range(len(manifolds_list))))

NameError: name 'compute_causalities' is not defined

In [68]:
manifolds_list[0]

[[array([-1.40642002e-01, -3.88161987e-01, -5.74181020e-01, -5.83769977e-01,
         -3.76219988e-01, -2.23350003e-02,  3.20762992e-01,  4.89035994e-01,
          4.01670992e-01,  1.08618997e-01, -2.34226003e-01, -4.48581994e-01,
         -4.26905990e-01, -1.81226999e-01,  1.77074999e-01,  5.07900000e-01,
          7.15746999e-01,  7.81772017e-01,  7.42668986e-01,  6.41453028e-01,
          4.91223991e-01,  2.78277993e-01, -2.29400001e-03, -3.11628014e-01,
         -5.65971971e-01, -6.76743984e-01, -6.06692016e-01, -4.02310997e-01,
         -1.76734999e-01, -5.03030010e-02, -8.48850012e-02, -2.52328992e-01,
         -4.54937011e-01, -5.82488000e-01, -5.69119990e-01, -4.17593002e-01,
         -1.82984993e-01,  6.57309964e-02,  2.77741998e-01,  4.28640991e-01,
          5.08812010e-01,  5.10424018e-01,  4.28748995e-01,  2.77258009e-01,
          1.00414999e-01, -3.34639996e-02, -6.25820011e-02,  3.10260002e-02,
          2.02867001e-01,  3.62749994e-01,  4.21303988e-01,  3.40101987e-01,

In [71]:
list(permutations(range(len(manifolds_list[0]))))[:3]